In [ ]:
import numpy as np
import pandas as pd
from math import floor, sqrt
from PIL import Image, ImageFilter, ImageDraw, ImageOps, ImageStat
from tqdm import tqdm
import os
import glob
import os

In [ ]:
username = 'your username' #username of account you want to mosaic

In [ ]:
!instagram-scraper username

In [ ]:
try:
    os.mkdir(os.getcwd()+'/black/')
    os.mkdir(os.getcwd()+'/target sections/')
    os.mkdir(os.getcwd()+'/filler sections/')
except OSError:
    print ("Creation of the directory failed")
else:
    print ("Successfully created directory")

In [ ]:
#removes mp4 files leaving only jpg
file_list = glob.glob(username + '/*.jpg') 
for file_path in tqdm(file_list):
    try:
        os.remove(file_path)
    except:
        print("Error while deleting file : ", file_path)

In [ ]:
#crop to square
def crop_max_square(pil_img):
    return crop_center(pil_img, min(pil_img.size), min(pil_img.size))


def crop_center(pil_img, crop_width, crop_height):
    img_width, img_height = pil_img.size
    return pil_img.crop(((img_width - crop_width) // 2,
                         (img_height - crop_height) // 2,
                         (img_width + crop_width) // 2,
                         (img_height + crop_height) // 2))

In [ ]:
file_list = glob.glob(username + '/*.jpg')
num = 0

#for i in tqdm(range(5)):  can use to create duplicate photos to raise resolution
for file in tqdm(file_list):
    im = Image.open(file)
    im = crop_max_square(im).convert('L')
    im.save('black/' + str(num) + ".jpg")
    num+=1

instruction: set image to mosaic as target.jpg in mosaic folder

In [ ]:
def get_mosaic_resolution():
    file_list = glob.glob('black/*.jpg')
    return floor(sqrt(len(file_list)))

In [ ]:
def get_target_resolution(mosaic_resolution):
    target_size = min(Image.open('target.jpg').size)
    return (target_size//mosaic_resolution)*mosaic_resolution

In [ ]:
mosaic_res =  get_mosaic_resolution()
resolution = get_target_resolution(mosaic_res)
section_size = resolution // mosaic_res

target = Image.open('target.jpg')
target = crop_max_square(target)
target = target.rotate(270).convert('L')


#cuts out sections from target and deposites in target sections folder
for i in range(mosaic_res**2):
    section = target.crop(((i*section_size)%resolution,floor(i/mosaic_res)*section_size ,(i*section_size)%resolution + section_size,floor(i/mosaic_res)*section_size+section_size))
    section.save('target sections/' + str(i) + '.jpg')


# makes thumbnails and deposits in filler sections folder
file_list = glob.glob('black/*.jpg')
for i, file in tqdm(enumerate(file_list[:mosaic_res**2])):
    im = Image.open(str(file))
    im.thumbnail((round(resolution / mosaic_res),round(resolution / mosaic_res)))
    im.save('filler sections/' + str(i) + ".jpg")
   


In [ ]:
def average(im):
    return ImageStat.Stat(im).sum[0]/ImageStat.Stat(im).count[0]

In [ ]:
def makeDictionary(filepath,num_items):
    brightness = {}
    for i in range(num_items+1):
        im = Image.open(filepath+str(i)+'.jpg')
        brightness[i]=average(im)
    return brightness

In [ ]:
list_filler = makeDictionary('filler sections/',(mosaic_res**2)-1)
list_target = makeDictionary('target sections/',(mosaic_res**2)-1)

sorted_filler = dict(sorted(list_filler.items(), key=lambda item: item[1]))
sorted_target = dict(sorted(list_target.items(), key=lambda item: item[1]))

#creates a dictionary with target segments as keys with the closest matching filler segment as value
ordered_pairs= {}
for i in tqdm(range(len(sorted_filler))):
    ordered_pairs[list(sorted_target)[i]]=list(sorted_filler)[i]


In [ ]:

def Print_mosaic(ordered_pairs):
    im_size = Image.open('filler sections/0.jpg').size[0]
    new_image = Image.new('L',(mosaic_res*im_size, mosaic_res*im_size))
  
    for i in ordered_pairs:
        im = Image.open('filler sections/' + str(ordered_pairs[i]) + '.jpg')
        new_image.paste(im,((i*section_size)%resolution,floor(i/mosaic_res)*section_size))

    transparent_overlay = target
    transparent_overlay.putalpha(140)
    new_image.paste(transparent_overlay,(0,0),transparent_overlay.convert('RGBA'))
    new_image.show()
    new_image.save('output.jpg')

In [ ]:
Print_mosaic(ordered_pairs)